In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
torch.cuda.is_available()

False

**Load Data and Preprocessing**


In [ ]:
import pandas as pd
df= pd.read_excel('/content/drive/MyDrive/dataset_with_image_name.xlsx')
df.head(10)

,Image_Name,Type,Color,Gender,color_normalized,color_normalized2,color_normalized3,type_normalized,Image_Name_Only
0,https://cdn.modiseh.com/unsafe/medium/nc1161/n...,تاپ,زرشکی,زنانه,قرمز,قرمز,قرمز,تی شرت,nc1161_1_-.JPG
1,https://cdn.modiseh.com/unsafe/medium/nc1162/n...,شلوار,زرشکی,زنانه,قرمز,قرمز,قرمز,شلوار,nc1162_1_-.JPG
2,https://cdn.modiseh.com/unsafe/medium/nc1163/n...,تاپ,سبز سدری,زنانه,سبز سدری,سبز سدری,سبز,تی شرت,nc1163_1_-.JPG
3,https://cdn.modiseh.com/unsafe/medium/nc1164/n...,دامن شلواری,سبز سدری,زنانه,سبز سدری,سبز سدری,سبز,پیراهن,nc1164_1_-.JPG
4,https://cdn.modiseh.com/unsafe/medium/nc1167/n...,مانتو,چند رنگ,زنانه,چند رنگ,چند رنگ,چند رنگ,مانتو,nc1167_1_-.jpg
5,https://cdn.modiseh.com/unsafe/medium/nc1168/n...,مانتو,مشکی,زنانه,مشکی,مشکی,مشکی,مانتو,nc1168_1_-.jpg
6,https://cdn.modiseh.com/unsafe/medium/nc1169/n...,مانتو,شیری صورتی,زنانه,شیری صورتی,شیری صورتی,سفید,مانتو,nc1169_1_-.jpg
7,https://cdn.modiseh.com/unsafe/medium/nc1170/n...,مانتو,شیری سبز,زنانه,شیری سبز,شیری سبز,سفید,مانتو,nc1170_1_-.jpg
8,https://cdn.modiseh.com/unsafe/medium/nc1171/n...,مانتو,سرمه ای,زنانه,سرمه ای,سرمه ای,سرمه ای,مانتو,nc1171_1_-.jpg
9,https://cdn.modiseh.com/unsafe/medium/nc1172/n...,شومیز,کرم,زنانه,کرم,کرم,کرم,پیراهن,nc1172_1_-.jpg


In [ ]:
def normalize_color_name(name):
    if not isinstance(name, str):
        return name

    name = name.strip()
    name = re.sub(r'\s+', ' ', name)
    name = name.replace('ـ', '')
    name = name.replace('ي', 'ی')
    name = name.replace('ك', 'ک')
    name = name.replace('ۀ', 'ه')
    name = name.replace('‌', ' ')
    name = name.replace('-', ' ')
    name = name.lower()

    name = name.replace("سورمه", "سرمه")
    name = name.replace("قهوه‌", "قهوه")
    name = name.replace("یشمی", "سبز")

    return name

df['color_normalized2'] = df['color_normalized'].apply(normalize_color_name)


In [ ]:
base_colors = {
    "مشکی": ["مشکی", "مشکی/.*", "مشکی .*", "دودی", "مدادی", "فیلی", "طوسی .*", "سرمه.*", "آبی تیره"],
    "سفید": ["سفید", "شیری", "استخوانی", "کرم .*", "سفید .*"],
    "طوسی": ["طوسی", "طوسی .*", "دودی .*", "مدادی", "فیلی"],
    "قرمز": ["قرمز", "زرشکی", "عنابی", "سرخابی", "بادمجانی", "نارنجی .*"],
    "صورتی": ["صورتی", "گلبهی", "شیری صورتی", "رزگلد", "هلویی .*"],
    "سبز": ["سبز", "سبز .*", "یشمی", "نعناعی", "سبز سدری", "سبز مغز پسته .*", "سبز روشن", "سبز تیره", "ارتشی", "سبز آجری"],
    "آبی": ["آبی", "آبی .*", "فیروزه .*", "نیلی"],
    "قهوه ای": ["قهوه", "کاکائویی", "نسکافه .*", "کرم قهوه .*"],
    "زرد": ["زرد", "لیمویی", "خردلی", "عسلی .*"],
    "بنفش": ["بنفش", "یاسی", "بادمجانی .*"],
    "نارنجی": ["نارنجی", "آجری", "طلایی .*"],
    "چند رنگ": ["چند رنگ", "چندرنگ", "رنگارنگ", "رنگی", "رنگنبدی .*"]
}
import re

def normalize_color(color_name):
    color_name = str(color_name).strip()
    for base, patterns in base_colors.items():
        for pat in patterns:
            if re.match(pat, color_name, flags=re.IGNORECASE):
                return base
    return "سایر"


df['color_normalized3'] = df['color_normalized2'].apply(normalize_color)

df.to_excel("colors_normalized.xlsx", index=False, engine='openpyxl')
print("✅ فایل colors_normalized.xlsx با رنگ‌های استاندارد ساخته شد!")


✅ فایل colors_normalized.xlsx با رنگ‌های استاندارد ساخته شد!


In [ ]:
import pandas as pd
df2= pd.read_excel('/content/drive/MyDrive/Normilized1.xlsx')
df2.head(3)

,Image_Name,Type,Color,Gender,color_normalized,color_normalized2,color_normalized3
0,https://cdn.modiseh.com/unsafe/medium/nc1161/n...,تاپ,زرشکی,زنانه,قرمز,قرمز,قرمز
1,https://cdn.modiseh.com/unsafe/medium/nc1162/n...,شلوار,زرشکی,زنانه,قرمز,قرمز,قرمز
2,https://cdn.modiseh.com/unsafe/medium/nc1163/n...,تاپ,سبز سدری,زنانه,سبز سدری,سبز سدری,سبز


In [ ]:

type_mapping = {
    "تی شرت": ["تی شرت", "تیشرت", "تی شرت ورزشی", "تاپ ورزشی", "تاپ", "تاپ نیم تنه", "کراپ تاپ", "ست تی شرت و شلوار", "ست تی شرت و شلوارک"],
    "پیراهن": ["پیراهن", "پیراهن ورزشی", "شومیز", "بلوز", "پولوشرت", "پولوشرت ورزشی", "ست شومیز و شلوار", "ست پیراهن و شلوارک", "تونیک", "دامن شلواری"],
    "شلوار": ["شلوار", "شلوار جین", "شلوارک", "شلوار اسلش", "شلوار راحتی", "شلوار پارچه ای", "شلوار ورزشی", "لگینگ", "لگینگ ورزشی", "ست نیم تنه و شلوارک", "ست نیم تنه و شلوار", "ست تاپ و شلوارک", "ست بلوز و شلوار", "ست بلوز و شلوارک", "ست شومیز و شلوار", "ست کراوات و پوشت"],
    "کفش": ["کفش", "کفش کژوال", "کفش ورزشی", "پاشنه بلند", "بوت", "ونس", "کفش رسمی", "اسنیکرز", "کفش تخت", "اکسسوری کفش"],
    "کیف": ["کیف", "کیف دوشی", "کیف پول", "کیف اداری", "کیف کمری", "کیف مدارک", "کیف تبلت", "کوله پشتی", "کوله پشتی لپ تاپ", "ست کیف و کفش", "کیف ورزشی"],
    "مانتو": ["مانتو", "ست مانتو و شلوار"],
    "ژاکت و سویشرت": ["ژاکت", "سویشرت", "هودی", "پلیور و ژاکت", "ست سویشرت و شلوار", "سوییشرت", "سویشرت ورزشی", "هودی ورزشی", "پولوشرت ورزشی"],
    "لباس زیر": ["سوتین", "شورت", "شورت و سوتین", "زیرپوش", "اکسسوری لباس زیر", "شورتکس", "ست نیم تنه و شورت"],
    "لباس راحتی و خواب": ["لباس", "لباس خواب", "لباس راحتی منزل", "مایو", "مایو زنانه", "سرهمی", "اورال"],
    "اکسسوری": ["کمربند", "ساعت مچی", "دستبند", "گردنبند", "کلاه", "کلاه کپ", "جاکارتی", "جاکلیدی", "دستمال سر و گردن", "دستمال گردن", "تل ورزشی", "گیره شال و روسری", "پاپیون"],
    "چتر": ["چتر"],
    "پوشاک ست": ["ست", "ست تاپ و شلوارک", "ست نیم تنه و شلوار", "ست تی شرت و شلوار", "ست تی شرت و شلوارک", "ست کراوات و پوشت", "ست بلوز و شلوار", "ست بلوز و شلوارک", "ست شومیز و شلوار", "ست مانتو و شلوار", "ست پولوشرت و شلوار", "ست کت و سرهمی", "ست کت و دامن", "ست ورزشی", "ست هدیه"],
    "سایر": ["نامشخص", "مطابق تصویر"]
}


def normalize_type(type_name):
    if not isinstance(type_name, str):
        return "سایر"
    type_name = type_name.strip()
    for main_type, sub_types in type_mapping.items():
        if type_name in sub_types:
            return main_type
    return "سایر"


df2['type_normalized'] = df2['Type'].apply(normalize_type)

df2.to_excel("type_normalized.xlsx", index=False)

df2.head()


,Image_Name,Type,Color,Gender,color_normalized,color_normalized2,color_normalized3,type_normalized
0,https://cdn.modiseh.com/unsafe/medium/nc1161/n...,تاپ,زرشکی,زنانه,قرمز,قرمز,قرمز,تی شرت
1,https://cdn.modiseh.com/unsafe/medium/nc1162/n...,شلوار,زرشکی,زنانه,قرمز,قرمز,قرمز,شلوار
2,https://cdn.modiseh.com/unsafe/medium/nc1163/n...,تاپ,سبز سدری,زنانه,سبز سدری,سبز سدری,سبز,تی شرت
3,https://cdn.modiseh.com/unsafe/medium/nc1164/n...,دامن شلواری,سبز سدری,زنانه,سبز سدری,سبز سدری,سبز,پیراهن
4,https://cdn.modiseh.com/unsafe/medium/nc1165/n...,ست تاپ و شلوار,چند رنگ,زنانه,چند رنگ,چند رنگ,چند رنگ,سایر


**Data augmentation**

In [ ]:
import os
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tqdm import tqdm

image_folder = "/content/drive/MyDrive/images/"
output_folder = "/content/drive/MyDrive/image_augmented/"
AUGMENT_THRESHOLD = 1000
AUGMENT_PER_IMAGE = 3




datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.15,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)

def augment_images_for_class(df_class, label_name, output_dir, target_count=AUGMENT_THRESHOLD):
    current_count = len(df_class)
    needed = max(0, target_count - current_count)
    if needed <= 0:
        return

    os.makedirs(output_dir, exist_ok=True)
    print(f"Augmenting '{label_name}': {current_count} → {target_count} samples")

    for img_name in tqdm(df_class['Image_Name_Only'], desc=f"Augmenting {label_name}"):
        if needed <= 0:
            break

        filename = os.path.basename(img_name)
        img_path = os.path.join(image_folder, filename)
        if not os.path.exists(img_path):
            continue

        img = load_img(img_path, target_size=(224,224))
        x = img_to_array(img)
        x = x.reshape((1,) + x.shape)

        i = 0
        for batch in datagen.flow(x, batch_size=1,
                                  save_to_dir=output_dir,
                                  save_prefix='aug',
                                  save_format='jpg'):
            i += 1
            needed -= 1
            if i >= AUGMENT_PER_IMAGE or needed <= 0:
                break

# Step 1: Augment color classes
color_counts = df['color_normalized2'].value_counts().to_dict()
for color, count in color_counts.items():
    if count < AUGMENT_THRESHOLD:
        df_color_class = df[df['color_normalized2'] == color]
        output_dir = os.path.join(output_folder, "colors", color)
        augment_images_for_class(df_color_class, color, output_dir)

# Step 2: Augment type classes
type_counts = df['type_normalized'].value_counts().to_dict()
for t, count in type_counts.items():
    if count < AUGMENT_THRESHOLD:
        df_type_class = df[df['type_normalized'] == t]
        output_dir = os.path.join(output_folder, "types", t)
        augment_images_for_class(df_type_class, t, output_dir)

print("Augmentation completed.")


Exception ignored in: <function _xla_gc_callback at 0x7d47b70ed4e0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/jax/_src/lib/__init__.py", line 96, in _xla_gc_callback
    def _xla_gc_callback(*args):
    
KeyboardInterrupt: 


Augmenting 'آبی تیره': 868 → 1000 samples


Augmenting آبی تیره:   1%|          | 8/868 [00:01<03:24,  4.20it/s]


KeyboardInterrupt: 

**Model Training**

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.preprocessing import LabelEncoder
from PIL import Image
from sklearn.model_selection import train_test_split

df = pd.read_excel("/content/drive/MyDrive/combined_dataset.xlsx")

df['color_label'] = LabelEncoder().fit(df['color_normalized3'].fillna('unknown')).transform(df['color_normalized3'].fillna('unknown'))
df['type_label']  = LabelEncoder().fit(df['type_normalized'].fillna('unknown')).transform(df['type_normalized'].fillna('unknown'))

color_le = LabelEncoder().fit(df['color_normalized3'].fillna('unknown'))
type_le  = LabelEncoder().fit(df['type_normalized'].fillna('unknown'))

num_colors = len(color_le.classes_)
num_types  = len(type_le.classes_)
print("Num color classes:", num_colors)
print("Num type classes:", num_types)


Num color classes: 15
Num type classes: 11


In [3]:
class MultiLabelDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = row['Image_Path']
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)

        color_label = torch.tensor(row['color_label'], dtype=torch.long)
        type_label  = torch.tensor(row['type_label'], dtype=torch.long)
        return image, color_label, type_label


In [4]:
# Transforms
train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

# Split dataset
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

train_dataset = MultiLabelDataset(train_df, transform=train_transform)
val_dataset   = MultiLabelDataset(val_df, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Base model
base_model = models.efficientnet_b0(pretrained=True)
in_features = base_model.classifier[1].in_features
base_model.classifier = nn.Identity()  # remove original classifier
base_model = base_model.to(device)

# Freeze backbone
for param in base_model.parameters():
    param.requires_grad = False

# Separate heads
color_head = nn.Linear(in_features, num_colors).to(device)
type_head  = nn.Linear(in_features, num_types).to(device)

#  Loss & Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(list(base_model.parameters()) + list(color_head.parameters()) + list(type_head.parameters()), lr=1e-3)

# Mixed precision
scaler = torch.cuda.amp.GradScaler()

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/tmp/ipython-input-1511092517.py:24: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


In [ ]:
from tqdm import tqdm
epochs = 6

for epoch in range(epochs):
    base_model.train()
    total_loss = 0
    total_color_acc = 0
    total_type_acc = 0

    loop = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{epochs}]", leave=True)
    for imgs, color_labels, type_labels in loop:
        imgs = imgs.to(device)
        color_labels = color_labels.to(device)
        type_labels = type_labels.to(device)

        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
            features = base_model(imgs)
            color_logits = color_head(features)
            type_logits = type_head(features)
            loss_color = criterion(color_logits, color_labels)
            loss_type  = criterion(type_logits, type_labels)
            loss = loss_color + loss_type

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()
        total_color_acc += (color_logits.argmax(1) == color_labels).sum().item()
        total_type_acc  += (type_logits.argmax(1) == type_labels).sum().item()

        loop.set_postfix(
            Loss=total_loss/len(loop),
            Color_Acc=total_color_acc/len(train_dataset),
            Type_Acc=total_type_acc/len(train_dataset)
        )


    # Validation after each epoch

    base_model.eval()
    val_color_acc = 0
    val_type_acc  = 0
    with torch.no_grad():
        for imgs, color_labels, type_labels in val_loader:
            imgs = imgs.to(device)
            color_labels = color_labels.to(device)
            type_labels  = type_labels.to(device)
            features = base_model(imgs)
            color_logits = color_head(features)
            type_logits = type_head(features)
            val_color_acc += (color_logits.argmax(1) == color_labels).sum().item()
            val_type_acc  += (type_logits.argmax(1) == type_labels).sum().item()
    print(f"Validation Color Acc: {val_color_acc/len(val_dataset):.4f}, Type Acc: {val_type_acc/len(val_dataset):.4f}")

# -----------------------------
# 7️⃣ Save model
# -----------------------------
torch.save({
    'base_model_state_dict': base_model.state_dict(),
    'color_head_state_dict': color_head.state_dict(),
    'type_head_state_dict': type_head.state_dict(),
}, "/content/drive/MyDrive/fine_tuned_color_type_model.pth")


Epoch [1/6]:   0%|          | 0/1166 [00:00<?, ?it/s]/tmp/ipython-input-559461700.py:18: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch [1/6]: 100%|██████████| 1166/1166 [54:54<00:00,  2.83s/it, Color_Acc=0.626, Loss=1.92, Type_Acc=0.775]


Validation Color Acc: 0.7025, Type Acc: 0.8638


Epoch [2/6]: 100%|██████████| 1166/1166 [05:38<00:00,  3.44it/s, Color_Acc=0.694, Loss=1.45, Type_Acc=0.835]


Validation Color Acc: 0.7355, Type Acc: 0.8785


Epoch [3/6]: 100%|██████████| 1166/1166 [05:41<00:00,  3.41it/s, Color_Acc=0.71, Loss=1.34, Type_Acc=0.851]


...

Validation Color Acc: 0.7589, Type Acc: 0.8978


Epoch [5/6]: 100%|██████████| 1166/1166 [05:45<00:00,  3.37it/s, Color_Acc=0.731, Loss=1.22, Type_Acc=0.866]


Validation Color Acc: 0.7593, Type Acc: 0.9019


Epoch [6/6]: 100%|██████████| 1166/1166 [05:52<00:00,  3.31it/s, Color_Acc=0.734, Loss=1.19, Type_Acc=0.87]


Validation Color Acc: 0.7731, Type Acc: 0.9052


In [ ]:
import torch
from torchvision import transforms, models
from PIL import Image
import torch.nn as nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Label Encoders (must match training)
import pandas as pd
from sklearn.preprocessing import LabelEncoder

df = pd.read_excel("/content/drive/MyDrive/combined_dataset.xlsx")
color_le = LabelEncoder().fit(df['color_normalized3'].fillna('unknown'))
type_le  = LabelEncoder().fit(df['type_normalized'].fillna('unknown'))

num_colors = len(color_le.classes_)
num_types  = len(type_le.classes_)

# Load model
base_model = models.efficientnet_b0(pretrained=False)
in_features = base_model.classifier[1].in_features
base_model.classifier = nn.Identity()
base_model = base_model.to(device)

color_head = nn.Linear(in_features, num_colors).to(device)
type_head  = nn.Linear(in_features, num_types).to(device)

# Load weights
checkpoint = torch.load("/content/drive/MyDrive/fine_tuned_color_type_model.pth", map_location=device)
base_model.load_state_dict(checkpoint['base_model_state_dict'])
color_head.load_state_dict(checkpoint['color_head_state_dict'])
type_head.load_state_dict(checkpoint['type_head_state_dict'])

base_model.eval()
color_head.eval()
type_head.eval()

# Preprocessing for a new image
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

def predict_image(img_path):
    image = Image.open(img_path).convert('RGB')
    img_tensor = transform(image).unsqueeze(0).to(device)  # add batch dim

    with torch.no_grad():
        features = base_model(img_tensor)
        color_logits = color_head(features)
        type_logits  = type_head(features)

        color_idx = torch.argmax(color_logits, dim=1).item()
        type_idx  = torch.argmax(type_logits, dim=1).item()

    color_label = color_le.inverse_transform([color_idx])[0]
    type_label  = type_le.inverse_transform([type_idx])[0]

    return color_label, type_label

# Example usage
img_path = "/content/drive/MyDrive/images/nc1161_1_-.JPG"
pred_color, pred_type = predict_image(img_path)
print(f"Predicted color: {pred_color}, Predicted type: {pred_type}")


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


KeyboardInterrupt: 

In [14]:
from torchvision import transforms, models
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])
def predict_image(img_path):
    image = Image.open(img_path).convert('RGB')
    img_tensor = transform(image).unsqueeze(0).to(device)  # add batch dim

    with torch.no_grad():
        features = base_model(img_tensor)
        color_logits = color_head(features)
        type_logits  = type_head(features)

        color_idx = torch.argmax(color_logits, dim=1).item()
        type_idx  = torch.argmax(type_logits, dim=1).item()

    color_label = color_le.inverse_transform([color_idx])[0]
    type_label  = type_le.inverse_transform([type_idx])[0]

    return color_label, type_label

In [15]:
import os
input_folder = "/content/drive/MyDrive/input2/"
# Loop over all images in the folder
for img_file in os.listdir(input_folder):
    if img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
        img_path = os.path.join(input_folder, img_file)
        color, type_ = predict_image(img_path)
        print(f"{img_file} -> Predicted Color: {color}, Predicted Type: {type_}")


1111222.png -> Predicted Color: کرم, Predicted Type: تی شرت
213113.png -> Predicted Color: طوسی, Predicted Type: مانتو
wew3e3.png -> Predicted Color: سرمه ای, Predicted Type: شلوار


In [7]:
all_color_preds = []
all_color_labels = []
all_type_preds  = []
all_type_labels = []

base_model.eval()
color_head.eval()
type_head.eval()

with torch.no_grad():
    for imgs, color_labels, type_labels in val_loader:
        imgs = imgs.to(device)
        color_labels = color_labels.to(device)
        type_labels = type_labels.to(device)

        features = base_model(imgs)
        color_logits = color_head(features)
        type_logits  = type_head(features)

        color_pred = torch.argmax(color_logits, dim=1)
        type_pred  = torch.argmax(type_logits, dim=1)

        all_color_preds.extend(color_pred.cpu().numpy())
        all_color_labels.extend(color_labels.cpu().numpy())
        all_type_preds.extend(type_pred.cpu().numpy())
        all_type_labels.extend(type_labels.cpu().numpy())


In [ ]:
from sklearn.metrics import f1_score, accuracy_score, classification_report
f1_color = f1_score(all_color_labels, all_color_preds, average='weighted')
f1_type  = f1_score(all_type_labels, all_type_preds, average='weighted')

In [ ]:
print("Color Classification Report:\n")
print(classification_report(all_color_labels, all_color_preds, target_names=color_le.classes_))

print("\nType Classification Report:\n")
print(classification_report(all_type_labels, all_type_preds, target_names=type_le.classes_))


Color Classification Report:

              precision    recall  f1-score   support

     unknown       0.89      0.87      0.88       224
         آبی       0.67      0.77      0.72       667
        بنفش       0.75      0.86      0.80       201
         زرد       0.90      0.81      0.85       192
         سبز       0.87      0.76      0.81       878
     سرمه ای       0.76      0.69      0.72       928
        سفید       0.78      0.74      0.76       923
       صورتی       0.83      0.68      0.75       243
        طوسی       0.66      0.60      0.63       701
        قرمز       0.80      0.88      0.84       456
     قهوه ای       0.75      0.73      0.74       405
        مشکی       0.78      0.85      0.81      2397
      نارنجی       0.90      0.74      0.81       204
     چند رنگ       0.78      0.83      0.81       201
         کرم       0.75      0.78      0.76       704

    accuracy                           0.77      9324
   macro avg       0.79      0.77      0.78      9

**Fine-Tuning**

In [5]:
import torch
import torch.nn as nn
from torchvision import models
from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load backbone and heads (same as before)
base_model = models.efficientnet_b0(pretrained=True)
in_features = base_model.classifier[1].in_features
base_model.classifier = nn.Identity()
base_model = base_model.to(device)

color_head = nn.Linear(in_features, num_colors).to(device)
type_head  = nn.Linear(in_features, num_types).to(device)

# Load previous fine-tuned weights if you want to continue
checkpoint = torch.load("/content/drive/MyDrive/fine_tuned_color_type_model.pth", map_location=device)
base_model.load_state_dict(checkpoint['base_model_state_dict'], strict=False)
color_head.load_state_dict(checkpoint['color_head_state_dict'])
type_head.load_state_dict(checkpoint['type_head_state_dict'])

# Freeze everything first
for param in base_model.parameters():
    param.requires_grad = False

# Unfreeze last few layers for fine-tuning
# EfficientNet-B0 features: blocks[0] ... blocks[7]
# Let's unfreeze last 3 blocks + batchnorm
for idx in range(5, 8):  # blocks 5,6,7
    for param in base_model.features[idx].parameters():
        param.requires_grad = True

# Also unfreeze final BatchNorm layers if needed
for param in base_model.features[8:].parameters():  # usually final layers after blocks
    param.requires_grad = True

# -----------------------------
# Optimizer with lower learning rate for fine-tuning
# -----------------------------
optimizer = torch.optim.Adam(
    list(filter(lambda p: p.requires_grad, base_model.parameters())) +
    list(color_head.parameters()) +
    list(type_head.parameters()),
    lr=1e-4  # lower than initial training
)

criterion = nn.CrossEntropyLoss()
scaler = torch.cuda.amp.GradScaler()

# -----------------------------
# Training loop for fine-tuning
# -----------------------------
from tqdm import tqdm

epochs = 5

for epoch in range(epochs):
    base_model.train()
    total_loss = 0
    total_color_acc = 0
    total_type_acc = 0

    loop = tqdm(train_loader, desc=f"Fine-tune Epoch [{epoch+1}/{epochs}]")
    for imgs, color_labels, type_labels in loop:
        imgs = imgs.to(device)
        color_labels = color_labels.to(device)
        type_labels = type_labels.to(device)

        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            features = base_model(imgs)
            color_logits = color_head(features)
            type_logits  = type_head(features)
            loss_color = criterion(color_logits, color_labels)
            loss_type  = criterion(type_logits, type_labels)
            loss = loss_color + loss_type

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()
        total_color_acc += (color_logits.argmax(1) == color_labels).sum().item()
        total_type_acc  += (type_logits.argmax(1) == type_labels).sum().item()

        loop.set_postfix(
            Loss=total_loss/len(loop),
            Color_Acc=total_color_acc/len(train_dataset),
            Type_Acc=total_type_acc/len(train_dataset)
        )

torch.save({
    'base_model_state_dict': base_model.state_dict(),
    'color_head_state_dict': color_head.state_dict(),
    'type_head_state_dict': type_head.state_dict()
}, "/content/drive/MyDrive/fine_tuned_color_type_backbone.pth")

print("✅ Fine-tuning complete and model saved.")


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth


100%|██████████| 20.5M/20.5M [00:00<00:00, 169MB/s]
/tmp/ipython-input-4115331263.py:53: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
Fine-tune Epoch [1/5]:   0%|          | 0/1166 [00:00<?, ?it/s]/tmp/ipython-input-4115331263.py:75: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Fine-tune Epoch [5/5]: 100%|██████████| 1166/1166 [06:56<00:00,  2.80it/s, Color_Acc=0.948, Loss=0.194, Type_Acc=0.991]


✅ Fine-tuning complete and model saved.


In [8]:
from sklearn.metrics import f1_score, accuracy_score, classification_report


f1_color = f1_score(all_color_labels, all_color_preds, average='weighted')
f1_type  = f1_score(all_type_labels, all_type_preds, average='weighted')

print(f"F1 Score - Color: {f1_color:.4f}")
print(f"F1 Score - Type : {f1_type:.4f}")

F1 Score - Color: 0.9607
F1 Score - Type : 0.9946
